Please note this just example usage of Model Registry Python code, not desciption of all the Python framework that Model Registry offers. 

## Installation
Before we can do any work with Model Registry make sure that Model registry is installed and working. The very first thing we need to do we need load the Model Registry python library 

In [ ]:
%pip install --pre model-registry
import os
import boto3
import urllib3

## Set up Access to S3 for storage
Configure the following for storing the models in S3 bucket. Currently Model Registry does not have any storage built into it, it is a index/metadata store only. providing default storage from Model Registry is on the roadmap in future versions.

In [18]:
key_id = os.getenv("AWS_ACCESS_KEY_ID")
secret_key = os.getenv("AWS_SECRET_ACCESS_KEY")
region = os.getenv("AWS_DEFAULT_REGION")
endpoint = os.getenv("AWS_S3_ENDPOINT")
bucket_name = os.getenv("AWS_S3_BUCKET")

In [19]:
workbenche_bucket = boto3.client(
    "s3",
    region,
    aws_access_key_id=key_id,
    aws_secret_access_key=secret_key,
    endpoint_url=endpoint,
    use_ssl=True,
    verify=False
)

In [20]:
urllib3.disable_warnings()

## List Files in the Bucket

In [ ]:
object = workbenche_bucket.list_objects_v2(Bucket=bucket_name)
for obj in object["Contents"]:
    print(obj["Key"])

## Import and Connect to the Model Registry
The next is to import the Python library and connect to Model Registry before you can "register" a model in Model Registry. Note that there can be more than one Model Registry in given Open Data Hub cluster with varying degrees of access. You may or may not have access to all the Model Registries that are available. Some could be public registries that are available for all the users and some may be available for certain business units or teams. Your access to given registry is controlled by permissions that are granted to you. If you see any access permissions please check your permissions.

First step is to find Model Registry URL to connect to, use below to find the URL

## Connecting to Model Registry
You can connect to a secure Model Registry using the default constructor (recommended):

In [ ]:
from model_registry import ModelRegistry

# connect to the Model Registry
#registry = ModelRegistry("<MR-HOST>", author="<USER>", user_token="<TOKEN>")

registry = ModelRegistry("https://my-model-registry-rest.apps.ocp4.example.com", author="admin", user_token="sha256~ipeKcU2_xmFLs8R1MZ96lU0Is3uGIZr3MLLQimf5s")

# Registering models
Use the following code to register your model in the Model Registry. Once registered, the model can be shared across teams and deployed for inference.

When a model is stored in the Model Registry, it is assigned a name along with a version number that corresponds to the specific instance of the model. If you make updates or modifications to the model and create a new version, you can simply register this new version under the same model name.

To uniquely identify each version of the model, it’s common for developers to use a unique identifier, such as a commit ID, that reflects the changes made in that version. The metadata can be any arbitary data that can help describe the model  

In [27]:
from model_registry import utils
s3_file_name = "models/model-latest.onnx" 
model = registry.register_model(
    name="roey-model",                          # model name
    uri=utils.s3_uri_from(path=s3_file_name, endpoint=endpoint, bucket=bucket_name, region=region),
    version="1.0.0",                       # model version
    description="This is test model for demonstration purposes", # description of the model
    model_format_name="onnx",              # model format
    model_format_version="1",              # model format version
    storage_key="workbenche_bucket",      # S3 connection details
    metadata={                             # look at "introduction" on types of metadata to capture 
        "int_key": 1,                      
        "bool_key": False,
        "float_key": 3.14,
        "str_key": "str_value",
    }
)

In [ ]:

# add labels
model.custom_properties = {
    "production": "",
    "finance": "",
}
registry.update(model)

# view the model added to the registry
model = registry.get_registered_model("roey-model")
print(model)

version = registry.get_model_version("roey-model", "1.0.0")
print(version)